In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import nn, optim
from sklearn.metrics import precision_score, recall_score
from tqdm import tqdm
import os

In [3]:
# 1. Set the device (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Define data transformations (including augmentation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.RandomHorizontalFlip(),  # Augment with random horizontal flip
    transforms.RandomRotation(30),      # Augment with random rotation (30 degrees)
    transforms.ToTensor(),             # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

# 3. Define the dataset directories
train_dir = "./dataset_train/"  # Path to the training dataset
test_dir = "./dataset_test/"    # Path to the testing dataset

# 4. Load the training and testing datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=os.path.join(train_dir, ''), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(test_dir, ''), transform=transform)

# 5. Define DataLoader for training and testing with batching and shuffling for training
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

# 6. Print the length of the datasets
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")


Number of training samples: 8010
Number of testing samples: 2450


In [4]:
# 5. Load the pre-trained ResNet50 model and modify it for binary classification
model = torchvision.models.resnet50(pretrained=True)  # Load pre-trained ResNet50
model.fc = nn.Linear(model.fc.in_features, 2)  # Modify the final layer for binary classification (2 classes: REAL, FAKE)

# Move the model to the appropriate device (GPU/CPU)
model.to(device)

# 6. Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.0001)
class_weights = torch.tensor([2.0, 1.0]).to(device)  # Assign higher weight to FAKE
criterion = nn.CrossEntropyLoss(weight=class_weights)


# 7. Training loop (with Precision and Recall calculation at each epoch)
num_epochs = 7
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    all_preds = []
    all_labels = []

    # Loop through the training data
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", ncols=100):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        running_loss += loss.item()

        # Get predictions and store them
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())  # Convert to numpy for scikit-learn
        all_labels.extend(labels.cpu().numpy())

    # Calculate precision and recall after each epoch
    precision = precision_score(all_labels, all_preds, average='binary', pos_label=1)  # FAKE class
    recall = recall_score(all_labels, all_preds, average='binary', pos_label=1)  # FAKE class

    # Print the loss and metrics for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")
    print(f"Precision (FAKE class): {precision:.4f}")
    print(f"Recall (FAKE class): {recall:.4f}")


c:\Users\MSI\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\MSI\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/7: 100%|██████████████████████████████████████████████████| 501/501 [04:00<00:00,  2.08it/s]


Epoch 1/7, Loss: 0.3032
Precision (FAKE class): 0.8986
Recall (FAKE class): 0.7848


Epoch 2/7: 100%|██████████████████████████████████████████████████| 501/501 [04:02<00:00,  2.06it/s]


Epoch 2/7, Loss: 0.2099
Precision (FAKE class): 0.9359
Recall (FAKE class): 0.8646


Epoch 3/7: 100%|██████████████████████████████████████████████████| 501/501 [03:57<00:00,  2.11it/s]


...

Epoch 5/7, Loss: 0.1523
Precision (FAKE class): 0.9555
Recall (FAKE class): 0.9129


Epoch 6/7: 100%|██████████████████████████████████████████████████| 501/501 [03:54<00:00,  2.14it/s]


Epoch 6/7, Loss: 0.1459
Precision (FAKE class): 0.9602
Recall (FAKE class): 0.9152


Epoch 7/7: 100%|██████████████████████████████████████████████████| 501/501 [03:54<00:00,  2.14it/s]

Epoch 7/7, Loss: 0.1290
Precision (FAKE class): 0.9658
Recall (FAKE class): 0.9228


In [5]:
model.eval()  # Set to evaluation mode
with torch.no_grad():  # No gradient calculation during inference
    outputs = model(image)  # Get the model's prediction
    
    softmax = torch.nn.Softmax(dim=1)
    probabilities = softmax(outputs)
    
    # Get the class with the highest probability
    _, predicted = torch.max(probabilities, 1)

prediction = 'REAL' if predicted.item() == 0 else 'FAKE'
print(f"Prediction: {prediction}")


NameError: name 'image' is not defined

In [6]:
# 9. Save the trained model
torch.save(model.state_dict(), 'model/trained_resnet50.pth')
print("Model saved!")

Model saved!
